### WORK IN PROGRESS

Rotation vs Angle difference?!
We likely don't need rotation in the sphere, but rather subtraction of the angles.

Explanation for issue? As soon as reference is rotated onto forward vector, it is possible to rotate sphere around reference vector, potentially moving input direction up or down. 
So use subtraction of the angles. This keeps the up component of both vectors equal. 

In [19]:
%matplotlib qt
# inline

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np
from scipy.spatial.transform import Rotation as R
import random
from tqdm import tqdm
import pandas as pd

from AngleHelpers import calculate_thetas_from_unity_coords, calculate_rotation_matrix, unity_pts_to_right_handed_cartesian_coords, \
                    right_handed_cartesian_coords_to_unity_pts, angle_between, angle_in_180s, cart2sph, sph2cart, deg2rad, rad2deg



In [20]:
def create_relative_direction(inp_dir_unity,ref_dir_unity,method="anglediff_sphere_coords",log_verbose=False):
    '''
    Calculate a direction vector that has the same offset from the Unity forward vector as an input direction vector has from a reference direction vector.
    inp_dir_unity: Vector of actual direction data.
    ref_dir_unity: Vector of reference the actual data should be taken relative to. 
    log_verbose: Log debug information.
    
    Methods: 
    - "anglediff": Take the difference between input and reference vector for horizontal angle and elevation angle 
                   and apply that difference to the Unity forward vector.
    - "anglediff_sphere_coords" : Same as "anglediff", but with sphere coordinates.
    - "unit_sphere_rotation": Rotate the unit sphere the directions are located in such that the reference vector will point 
                              towards the Unity forward vector.
    
    Due to the reference vector being rotatable around itself when pointing in the direction of the forward vector, 
    still yielding a valid solution, different final results exist. For the purpose of Unity directions, the "anglediff"/ "anglediff_sphere_coords" 
    approach seems to be most useful.
    
    The resulting relative direction vector will be in Unity coordinates. 
    '''
    
    # make sure np.array exists 
    inp_dir_unity = np.array(inp_dir_unity)
    ref_dir_unity = np.array(ref_dir_unity)
    forward_dir_unity = np.array([0,0,1])
    
    # To cartesian from Unity 
    inp_dir_cartesian = unity_pts_to_right_handed_cartesian_coords([inp_dir_unity])[0]
    ref_dir_cartesian = unity_pts_to_right_handed_cartesian_coords([ref_dir_unity])[0]
    forward_dir_cartesian = unity_pts_to_right_handed_cartesian_coords([forward_dir_unity])[0]

    # Debug Info
    if log_verbose:
        print("Method: " + method)
    
    if method == "anglediff":
        
        # Calculate thetas, can only use horizontal, since vertical depends on different plane than projected one 
        inp_horizontal, _ = calculate_thetas_from_unity_coords(inp_dir_unity,ret_degree=True)
        ref_horizontal, _ = calculate_thetas_from_unity_coords(ref_dir_unity,ret_degree=True)

        # Calculate vertical angle offset as angle between direction vector and projection on x,y plane (cartesian)
        inp_vertical = angle_between(inp_dir_cartesian,[inp_dir_cartesian[0],inp_dir_cartesian[1],0])
        ref_vertical = angle_between(ref_dir_cartesian,[ref_dir_cartesian[0],ref_dir_cartesian[1],0])

        # if z (cartesian) is negative, vertical angle is negative 
        if inp_dir_cartesian[2] < 0:
            inp_vertical *= -1
        if ref_dir_cartesian[2] < 0:
            ref_vertical *= -1

        # Calculate angle offsets 
        offset_horizontal = inp_horizontal - ref_horizontal
        offset_vertical = inp_vertical - ref_vertical

        # Debug Info 
        if log_verbose:
            print("Input horizontal: " + str(inp_horizontal))
            print("Reference horizontal: " + str(ref_horizontal))
            print("Offset horizontal: " + str(offset_horizontal)) # rotate around Unity y, Right-Handed cartesian z
            print("Input vertical: " + str(inp_vertical))
            print("Reference vertical: " + str(ref_vertical))
            print("Offset vertical: " + str(offset_vertical)) # rotate around Unity/ Right-Handed cartesian x 

        # calculate rot matrices
        # axis are for right-handed coordinate system
        # angle inverted for horizontal
        horizontal_rot_mat = calculate_rotation_matrix(rotate_around_axis="z", angle= -1 * offset_horizontal, angle_in_degree=True)
        vertical_rot_mat = calculate_rotation_matrix(rotate_around_axis="x", angle=offset_vertical, angle_in_degree=True)

        # apply calculated relative rotations to forward Unity vector (0,0,1) (in right-handed cartesian (0,1,0)) to get final direction 
        # first vertical, then horizontal! Rotation axis must align 
        final_dir_unity = forward_dir_unity
        final_dir_cartesian = unity_pts_to_right_handed_cartesian_coords([final_dir_unity])[0]
        final_dir_cartesian = np.dot(vertical_rot_mat, final_dir_cartesian)
        final_dir_cartesian = np.dot(horizontal_rot_mat,final_dir_cartesian)

        # Verify final direction has offset values 
        final_vertical = angle_between(final_dir_cartesian,[final_dir_cartesian[0],final_dir_cartesian[1],0]) # angle between vec and projection on xy-plane (cartesian)
        final_horizontal = angle_between([final_dir_cartesian[0],final_dir_cartesian[1],0],[0,1,0]) # angle between projection on xy-plane and forward vector (cartesian)
       
        # Debug Info 
        if log_verbose:
            print("---")
            print("Final horizontal (should match offset horizontal): " + str(final_horizontal)) 
            print("Final vertical (should match offset vertical): " + str(final_vertical))




    elif method == "anglediff_sphere_coords":
       
        # compute spherical coordinates 
        inp_dir_spherical = cart2sph(inp_dir_cartesian)
        ref_dir_spherical = cart2sph(ref_dir_cartesian) 
        forward_dir_spherical = cart2sph(forward_dir_cartesian)

        # compute difference between ref and input, modulo and add to forward dir 
        final_dir_spherical = (inp_dir_spherical[0] - ref_dir_spherical[0], inp_dir_spherical[1] - ref_dir_spherical[1], 1)
        final_dir_spherical = (final_dir_spherical[0] % (np.pi * 2), final_dir_spherical[1] % (np.pi * 2), 1)
        final_dir_spherical = (final_dir_spherical[0] + forward_dir_spherical[0], final_dir_spherical[1] + forward_dir_spherical[1],1)
        final_dir_spherical = (final_dir_spherical[0] % (np.pi * 2), final_dir_spherical[1] % (np.pi * 2), 1)

        # convert final direction back to cartesian coordinates
        final_dir_cartesian = sph2cart(*final_dir_spherical)

        # Debug Info
        if log_verbose:

            # verify vectors and directions 
            inp_vert_angle = angle_between(inp_dir_cartesian,[inp_dir_cartesian[0],inp_dir_cartesian[1],0]) # angle between vec and projection on xy-plane (cartesian)
            inp_horiz_angle = angle_between([inp_dir_cartesian[0],inp_dir_cartesian[1],0],[0,1,0]) # angle between projection on xy-plane and forward vector (cartesian)

            ref_vert_angle = angle_between(ref_dir_cartesian,[ref_dir_cartesian[0],ref_dir_cartesian[1],0]) # angle between vec and projection on xy-plane (cartesian)
            ref_horiz_angle = angle_between([ref_dir_cartesian[0],ref_dir_cartesian[1],0],[0,1,0]) # angle between projection on xy-plane and forward vector (cartesian)

            final_vert_angle = angle_between(final_dir_cartesian,[final_dir_cartesian[0],final_dir_cartesian[1],0]) # angle between vec and projection on xy-plane (cartesian)
            final_horiz_angle = angle_between([final_dir_cartesian[0],final_dir_cartesian[1],0],[0,1,0]) # angle between projection on xy-plane and forward vector (cartesian)

            # if z (cartesian) is negative, vertical angle is negative 
            if inp_dir_cartesian[2] < 0:
                inp_vert_angle *= -1
            if ref_dir_cartesian[2] < 0:
                ref_vert_angle *= -1
            if final_dir_cartesian[2] < 0:
                final_vert_angle *= -1

            # if x (cartesian) is negative, horizontal angle is negative 
            if inp_dir_cartesian[0] < 0:
                inp_horiz_angle *= -1
            if ref_dir_cartesian[0] < 0:
                ref_horiz_angle *= -1
            if final_dir_cartesian[0] < 0:
                final_horiz_angle *= -1

            # print verification infos 
            print("Horizontal angles:")
            print("\tInput: " + str(inp_horiz_angle))
            print("\tReference: " + str(ref_horiz_angle))
            print("\tFinal: " + str(final_horiz_angle))
            print("Vertical angles:")
            print("\tInput: " + str(inp_vert_angle))
            print("\tReference:" + str(ref_vert_angle))
            print("\tFinal: " + str(final_vert_angle))
            print("Vectors:")
            print("\tInput: " + str(inp_dir_cartesian))
            print("\tReference: " + str(ref_dir_cartesian))
            print("\tFinal: " + str(final_dir_cartesian))


        
    
    elif method == "unit_sphere_rotation":

        # Calculate horizontal angle offset as angle between direction vector's projection on x,y plane (cartesian) and forward vector
        # Alternatively, use calculate_thetas_from_unity_coords (but can only use horizontal value here, because projection would skew vertical value)
        ref_horizontal = angle_between([ref_dir_cartesian[0],ref_dir_cartesian[1],0],[0,1,0])

        # Calculate vertical angle offset as angle between direction vector and projection on x,y plane (cartesian)
        ref_vertical = angle_between(ref_dir_cartesian,[ref_dir_cartesian[0],ref_dir_cartesian[1],0])

        # if z (cartesian) is negative, vertical angle is negative 
        if ref_dir_cartesian[2] < 0:
            ref_vertical *= -1

        # if x (cartesian) is negative, horizontal angle is negative 
        if ref_dir_cartesian[0] < 0:
            ref_horizontal *= -1

        # Calculate horizontal angle offset for input 
        inp_horizontal = angle_between([inp_dir_cartesian[0],inp_dir_cartesian[1],0],[0,1,0])

        # Calculate vertical angle offset for input
        inp_vertical = angle_between(inp_dir_cartesian,[inp_dir_cartesian[0],inp_dir_cartesian[1],0])

        # if z (cartesian) is negative, vertical angle is negative 
        if inp_dir_cartesian[2] < 0:
            inp_vertical *= -1

        # if x (cartesian) is negative, horizontal angle is negative 
        if inp_dir_cartesian[0] < 0:
            inp_horizontal *= -1

        # Debug Info
        if log_verbose:
            print("Reference horizontal: " + str(ref_horizontal))
            print("Reference vertical: " + str(ref_vertical))
            print("Input horizontal: " + str(inp_horizontal))
            print("Input vertical: " + str(inp_vertical))

        # calculate rot matrices
        # axis are for right-handed coordinate system
        # angle inverted for vertical
        horizontal_rot_mat = calculate_rotation_matrix(rotate_around_axis="z", angle= ref_horizontal, angle_in_degree=True)
        vertical_rot_mat = calculate_rotation_matrix(rotate_around_axis="x", angle= -1 *ref_vertical, angle_in_degree=True)

        # Apply calculated rotations to reference direction, in order to rotate it to forward direction (Unity 0,0,1)
        # First horizontal, then vertical! Rotation axis must align 
        rotated_ref = ref_dir_cartesian.copy()
        rotated_ref = np.dot(horizontal_rot_mat,rotated_ref)
        rotated_ref = np.dot(vertical_rot_mat, rotated_ref)
        
        # Debug Info 
        if log_verbose:
            
            # Calculate horizontal angle offset of rotated reference
            rotated_ref_horizontal = angle_between([rotated_ref[0],rotated_ref[1],0],[0,1,0])

            # Calculate vertical angle offset of rotated reference
            rotated_ref_vertical = angle_between(rotated_ref,[rotated_ref[0],rotated_ref[1],0])

            # if z (cartesian) is negative, vertical angle is negative 
            if rotated_ref[2] < 0:
                rotated_ref_vertical *= -1

            # if x (cartesian) is negative, horizontal angle is negative 
            if ref_dir_cartesian[0] < 0:
                rotated_ref_horizontal *= -1

            print("Rotated reference: " + str(rotated_ref))
            print("Rotated reference horizontal: " + str(rotated_ref_horizontal))
            print("Rotated reference vertical: " + str(rotated_ref_vertical))


        # Apply calculated rotations to input direction
        # Same order, as for reference. First horizontal, then vertical! Rotation axis must align 
        rotated_inp = inp_dir_cartesian.copy()
        rotated_inp = np.dot(horizontal_rot_mat,rotated_inp)
        rotated_inp = np.dot(vertical_rot_mat, rotated_inp)

        # Debug Info
        if log_verbose:

            # Calculate horizontal angle offset of rotated input
            rotated_inp_horizontal = angle_between([rotated_inp[0],rotated_inp[1],0],[0,1,0])

            # Calculate vertical angle offset of rotated input
            rotated_inp_vertical = angle_between(rotated_inp,[rotated_inp[0],rotated_inp[1],0])

            # if z (cartesian) is negative, vertical angle is negative 
            if rotated_inp[2] < 0:
                rotated_inp_vertical *= -1

            # if x (cartesian) is negative, horizontal angle is negative 
            if rotated_inp[0] < 0:
                rotated_inp_horizontal *= -1

            print("Rotated input: " + str(rotated_inp))
            print("Rotated input horizontal: " + str(rotated_inp_horizontal))
            print("Rotated input vertical: " + str(rotated_inp_vertical))

        
        # Copy result to variable of same name as in other cases 
        final_dir_cartesian = rotated_inp
    
    else:
        print("Specified unknown method!")
        return
        
    
    # Make result a unit vector 
    final_dir_cartesian = final_dir_cartesian / np.linalg.norm(final_dir_cartesian)
    
    # Transform result to Unity coordinates 
    final_dir_unity = right_handed_cartesian_coords_to_unity_pts([final_dir_cartesian])[0]
   

    return final_dir_unity

   


def create_relative_directions(inp_unity_x,inp_unity_y,inp_unity_z,ref_unity_x,ref_unity_y,ref_unity_z,method="anglediff_sphere_coords"):
    '''
    Creates for a Pandas Series of Unity x,y,z inputs and references directions that are 
    relative to the Unity forward vector. 
    Methods: 
    - "anglediff"
    - "anglediff_sphere_coords" 
    - "unit_sphere_rotation"
    See create_relative_direction() for more infos. 
    
    Return Unity x,y,z series for the relative directions.
    '''
    
    
    # Progress bar 
    tqdm.pandas()
    
    # Create dataframe 
    dataframe = pd.DataFrame(columns=["inp_x","inp_y","inp_z","ref_x","ref_y","ref_z","final_x","final_y","final_z"])

    # Set data
    dataframe["inp_x"] = inp_unity_x
    dataframe["inp_y"] = inp_unity_y
    dataframe["inp_z"] = inp_unity_z
    dataframe["ref_x"] = ref_unity_x
    dataframe["ref_y"] = ref_unity_y
    dataframe["ref_z"] = ref_unity_z
    
    # Init final columns
    dataframe["final_x"] = np.NAN
    dataframe["final_y"] = np.NAN
    dataframe["final_z"] = np.NAN

    # Define method that is to be applied per row 
    def apply_transform(arg):

        # Calculate relative direction 
        result = create_relative_direction((arg["inp_x"],arg["inp_y"],arg["inp_z"]),(arg["ref_x"],arg["ref_y"],arg["ref_z"]),method,log_verbose=False)

        # Add result to row of dataframe
        arg["final_x"] = result[0]
        arg["final_y"] = result[1]
        arg["final_z"] = result[2]
        
        return arg


    # Apply transform 
    dataframe = dataframe.progress_apply(lambda row: apply_transform(row), axis=1)
    
    return dataframe["final_x"], dataframe["final_y"], dataframe["final_z"]
    
    

In [21]:
#### TODO: MOre testing! 

print(create_relative_directions([0,1],[0,1],[0,1],[1,0],[1,0],[1,0],method="anglediff_sphere_coords"))
   

100%|██████████| 2/2 [00:00<00:00,  7.49it/s]

(0    0.57735
1   -0.57735
Name: final_x, dtype: float64, 0   -0.57735
1    0.57735
Name: final_y, dtype: float64, 0    0.57735
1    0.57735
Name: final_z, dtype: float64)


## Using the unifying method

In [15]:
print("Using the unifying method")

fig = plt.figure(figsize=(15,10))
fig.canvas.manager.set_window_title("Using the unifying method")
ax1 = fig.add_subplot(1,1,1, projection='3d')

# Input and reference 
inp_dir_unity = np.array([-1,0,1])
ref_dir_unity = np.array([0,-1,-1])

# To cartesian from Unity 
inp_dir_cartesian = unity_pts_to_right_handed_cartesian_coords([inp_dir_unity])[0]
ref_dir_cartesian = unity_pts_to_right_handed_cartesian_coords([ref_dir_unity])[0]


# Get output 
final_dir_unity = create_relative_direction(inp_dir_unity,ref_dir_unity,method="anglediff_sphere_coords",log_verbose=False)
final_dir_cartesian = unity_pts_to_right_handed_cartesian_coords([final_dir_unity])[0]


# Plot
ax1.quiver(0,0,0,2,0,0,color="black") # axis
ax1.quiver(0,0,0,0,2,0,color="black") # axis
ax1.quiver(0,0,0,0,0,2,color="black") # axis
ax1.quiver(0,0,0,inp_dir_cartesian[0],inp_dir_cartesian[1],inp_dir_cartesian[2],color="green") # input
ax1.quiver(0,0,0,ref_dir_cartesian[0],ref_dir_cartesian[1],ref_dir_cartesian[2],color="blue") # reference
ax1.quiver(0,0,0,final_dir_cartesian[0],final_dir_cartesian[1],final_dir_cartesian[2],color="red") # final rotated

plt.xlim(-2.5, 2.5)
plt.ylim(-2.5, 2.5)
ax1.set_zlim(-2.5,2.5)
ax1.set_xlabel("Unity X / Cartesian X")
ax1.set_ylabel("Unity Z / Cartesian Y")
ax1.set_zlabel("Unity Y / Cartesian Z")

ax1.scatter(0,0,0,marker=".",color="black",s=500)

plt.show()


Using the unifying method


## Individual approaches

In [9]:
# %%timeit

# Version 2
# Basic idea
# Find rotations around z (cartesian, Unity y) and subsequently x (cartesian, Unity x) 
# that would rotate reference vector to forward direction (Unity 0,0,1). 
# Apply those rotations to reference vector, but also to input vector.
# Resulting rotated input vector is final direction relative to forward vector. 
print("Version 2 - Rotation of the unit sphere")

fig = plt.figure(figsize=(15,10))
fig.canvas.manager.set_window_title("Version 2 - Rotation of the unit sphere")
ax1 = fig.add_subplot(1,1,1, projection='3d')

# Input and reference 
#inp_dir_unity = np.array([1,-0.5,0.1])
#ref_dir_unity = np.array([-0.2,1,0.8])
#inp_dir_unity = np.array([-1,1.5,0.3])
#ref_dir_unity = np.array([-0.2,-1,0.8])
inp_dir_unity = np.array([-1,0,1])
ref_dir_unity = np.array([0,-1,-1])



# To cartesian from Unity 
inp_dir_cartesian = unity_pts_to_right_handed_cartesian_coords([inp_dir_unity])[0]
ref_dir_cartesian = unity_pts_to_right_handed_cartesian_coords([ref_dir_unity])[0]

# Calculate horizontal angle offset as angle between direction vector's projection on x,y plane (cartesian) and forward vector
# Alternatively, use calculate_thetas_from_unity_coords (but can only use horizontal value here, because projection would skew vertical value)
ref_horizontal = angle_between([ref_dir_cartesian[0],ref_dir_cartesian[1],0],[0,1,0])

# Calculate vertical angle offset as angle between direction vector and projection on x,y plane (cartesian)
ref_vertical = angle_between(ref_dir_cartesian,[ref_dir_cartesian[0],ref_dir_cartesian[1],0])

# if z (cartesian) is negative, vertical angle is negative 
if ref_dir_cartesian[2] < 0:
    ref_vertical *= -1
    
# if x (cartesian) is negative, horizontal angle is negative 
if ref_dir_cartesian[0] < 0:
    ref_horizontal *= -1
    
# Calculate horizontal angle offset for input 
inp_horizontal = angle_between([inp_dir_cartesian[0],inp_dir_cartesian[1],0],[0,1,0])

# Calculate vertical angle offset for input
inp_vertical = angle_between(inp_dir_cartesian,[inp_dir_cartesian[0],inp_dir_cartesian[1],0])

# if z (cartesian) is negative, vertical angle is negative 
if inp_dir_cartesian[2] < 0:
    inp_vertical *= -1
    
# if x (cartesian) is negative, horizontal angle is negative 
if inp_dir_cartesian[0] < 0:
    inp_horizontal *= -1

print("Reference horizontal: " + str(ref_horizontal))
print("Reference vertical: " + str(ref_vertical))
print("Input horizontal: " + str(inp_horizontal))
print("Input vertical: " + str(inp_vertical))

# calculate rot matrices
# axis are for right-handed coordinate system
# angle inverted for vertical
horizontal_rot_mat = calculate_rotation_matrix(rotate_around_axis="z", angle= ref_horizontal, angle_in_degree=True)
vertical_rot_mat = calculate_rotation_matrix(rotate_around_axis="x", angle= -1 *ref_vertical, angle_in_degree=True)


# Apply calculated rotations to reference direction, in order to rotate it to forward direction (Unity 0,0,1)
# First horizontal, then vertical! Rotation axis must align 
rotated_ref = ref_dir_cartesian.copy()
rotated_ref = np.dot(horizontal_rot_mat,rotated_ref)
rotated_ref = np.dot(vertical_rot_mat, rotated_ref)


# Calculate horizontal angle offset of rotated reference
rotated_ref_horizontal = angle_between([rotated_ref[0],rotated_ref[1],0],[0,1,0])

# Calculate vertical angle offset of rotated reference
rotated_ref_vertical = angle_between(rotated_ref,[rotated_ref[0],rotated_ref[1],0])

# if z (cartesian) is negative, vertical angle is negative 
if rotated_ref[2] < 0:
    rotated_ref_vertical *= -1
    
# if x (cartesian) is negative, horizontal angle is negative 
if ref_dir_cartesian[0] < 0:
    rotated_ref_horizontal *= -1

print("Rotated reference: " + str(rotated_ref))
print("Rotated reference horizontal: " + str(rotated_ref_horizontal))
print("Rotated reference vertical: " + str(rotated_ref_vertical))



# Apply calculated rotations to input direction
# Same order, as for reference. First horizontal, then vertical! Rotation axis must align 
rotated_inp = inp_dir_cartesian.copy()
rotated_inp = np.dot(horizontal_rot_mat,rotated_inp)
rotated_inp = np.dot(vertical_rot_mat, rotated_inp)


# Calculate horizontal angle offset of rotated input
rotated_inp_horizontal = angle_between([rotated_inp[0],rotated_inp[1],0],[0,1,0])

# Calculate vertical angle offset of rotated input
rotated_inp_vertical = angle_between(rotated_inp,[rotated_inp[0],rotated_inp[1],0])

# if z (cartesian) is negative, vertical angle is negative 
if rotated_inp[2] < 0:
    rotated_inp_vertical *= -1
    
# if x (cartesian) is negative, horizontal angle is negative 
if rotated_inp[0] < 0:
    rotated_inp_horizontal *= -1

print("Rotated input: " + str(rotated_inp))
print("Rotated input horizontal: " + str(rotated_inp_horizontal))
print("Rotated input vertical: " + str(rotated_inp_vertical))



# Plot
ax1.quiver(0,0,0,2,0,0,color="black") # axis
ax1.quiver(0,0,0,0,2,0,color="black") # axis
ax1.quiver(0,0,0,0,0,2,color="black") # axis
ax1.quiver(0,0,0,inp_dir_cartesian[0],inp_dir_cartesian[1],inp_dir_cartesian[2],color="green") # input
ax1.quiver(0,0,0,ref_dir_cartesian[0],ref_dir_cartesian[1],ref_dir_cartesian[2],color="blue") # reference
ax1.quiver(0,0,0,rotated_ref[0],rotated_ref[1],rotated_ref[2],color="lightblue") # reference rotated
ax1.quiver(0,0,0,rotated_inp[0],rotated_inp[1],rotated_inp[2],color="lightgreen") # input rotated

plt.xlim(-2.5, 2.5)
plt.ylim(-2.5, 2.5)
ax1.set_zlim(-2.5,2.5)
ax1.set_xlabel("Unity X / Cartesian X")
ax1.set_ylabel("Unity Z / Cartesian Y")
ax1.set_zlabel("Unity Y / Cartesian Z")

ax1.scatter(0,0,0,marker=".",color="black",s=500)

plt.show()

Version 2 - Rotation of the unit sphere
Reference horizontal: 180.0
Reference vertical: -45.00000000000001
Input horizontal: -45.00000000000001
Input vertical: 1.2074182697257333e-06
Rotated reference: [1.22464680e-16 1.41421356e+00 2.22044605e-16]
Rotated reference horizontal: 0.0
Rotated reference vertical: 0.0
Rotated input: [ 1.         -0.70710678 -0.70710678]
Rotated input horizontal: 125.26438968275465
Rotated input vertical: -30.000000000000004


In [4]:
# %%timeit

# Version 1b
# Same idea as 1a, but use sphere coordinates
print("Version 1b - Angle differences with sphere coordinates")


fig = plt.figure(figsize=(15,10))
fig.canvas.manager.set_window_title("Version 1b - Angle differences with sphere coordinates")
ax1 = fig.add_subplot(1,1,1, projection='3d')

# forward 
forward_dir_unity = np.array([0,0,1])

# inputs
#inp_dir_unity = np.array([1,1,1])
#ref_dir_unity = np.array([-1,-1,1])
inp_dir_unity = np.array([-1,0,1])
ref_dir_unity = np.array([0,-1,-1])

# generate random data as input and reference in (-1,1)
if False:
    inp_dir_unity = np.array([random.random() * 2 - 1 for _ in range(3)])
    ref_dir_unity = np.array([random.random() * 2 - 1 for _ in range(3)]) 

# compute cartesian right-handed representation 
inp_dir_cartesian = unity_pts_to_right_handed_cartesian_coords([inp_dir_unity])[0]
ref_dir_cartesian = unity_pts_to_right_handed_cartesian_coords([ref_dir_unity])[0]
forward_dir_cartesian = unity_pts_to_right_handed_cartesian_coords([forward_dir_unity])[0]

#### 
#inp_dir_cartesian = np.array([0.4609944825876826, 0.1321670315678083, -0.4489406317631861])
#ref_dir_cartesian = np.array([-0.20632662366938836, -0.36921655868324166, 0.36551083215510016])
#inp_dir_cartesian = np.array([0, 1, 1])
#ref_dir_cartesian = np.array([0, 0, -1])



# compute spherical coordinates 
inp_dir_spherical = cart2sph(inp_dir_cartesian)
ref_dir_spherical = cart2sph(ref_dir_cartesian) 
forward_dir_spherical = cart2sph(forward_dir_cartesian)

# compute difference between ref and input, modulo and add to forward dir 
final_dir_spherical = (inp_dir_spherical[0] - ref_dir_spherical[0], inp_dir_spherical[1] - ref_dir_spherical[1], 1)
final_dir_spherical = (final_dir_spherical[0] % (np.pi * 2), final_dir_spherical[1] % (np.pi * 2), 1)
final_dir_spherical = (final_dir_spherical[0] + forward_dir_spherical[0], final_dir_spherical[1] + forward_dir_spherical[1],1)
final_dir_spherical = (final_dir_spherical[0] % (np.pi * 2), final_dir_spherical[1] % (np.pi * 2), 1)


# convert final direction back to cartesian coordinates
final_dir_cartesian = sph2cart(*final_dir_spherical)

# verify vectors and directions 
inp_vert_angle = angle_between(inp_dir_cartesian,[inp_dir_cartesian[0],inp_dir_cartesian[1],0]) # angle between vec and projection on xy-plane (cartesian)
inp_horiz_angle = angle_between([inp_dir_cartesian[0],inp_dir_cartesian[1],0],[0,1,0]) # angle between projection on xy-plane and forward vector (cartesian)

ref_vert_angle = angle_between(ref_dir_cartesian,[ref_dir_cartesian[0],ref_dir_cartesian[1],0]) # angle between vec and projection on xy-plane (cartesian)
ref_horiz_angle = angle_between([ref_dir_cartesian[0],ref_dir_cartesian[1],0],[0,1,0]) # angle between projection on xy-plane and forward vector (cartesian)

final_vert_angle = angle_between(final_dir_cartesian,[final_dir_cartesian[0],final_dir_cartesian[1],0]) # angle between vec and projection on xy-plane (cartesian)
final_horiz_angle = angle_between([final_dir_cartesian[0],final_dir_cartesian[1],0],[0,1,0]) # angle between projection on xy-plane and forward vector (cartesian)

# if z (cartesian) is negative, vertical angle is negative 
if inp_dir_cartesian[2] < 0:
    inp_vert_angle *= -1
if ref_dir_cartesian[2] < 0:
    ref_vert_angle *= -1
if final_dir_cartesian[2] < 0:
    final_vert_angle *= -1

# if x (cartesian) is negative, horizontal angle is negative 
if inp_dir_cartesian[0] < 0:
    inp_horiz_angle *= -1
if ref_dir_cartesian[0] < 0:
    ref_horiz_angle *= -1
if final_dir_cartesian[0] < 0:
    final_horiz_angle *= -1

# print verification infos 
print("Horizontal angles:")
print("\tInput: " + str(inp_horiz_angle))
print("\tReference: " + str(ref_horiz_angle))
print("\tFinal: " + str(final_horiz_angle))
print("Vertical angles:")
print("\tInput: " + str(inp_vert_angle))
print("\tReference:" + str(ref_vert_angle))
print("\tFinal: " + str(final_vert_angle))
print("Vectors:")
print("\tInput: " + str(inp_dir_cartesian))
print("\tReference: " + str(ref_dir_cartesian))
print("\tFinal: " + str(final_dir_cartesian))


# Plot
ax1.quiver(0,0,0,2,0,0,color="black") # axis
ax1.quiver(0,0,0,0,2,0,color="black") # axis
ax1.quiver(0,0,0,0,0,2,color="black") # axis
ax1.quiver(0,0,0,inp_dir_cartesian[0],inp_dir_cartesian[1],inp_dir_cartesian[2],color="green") # input
ax1.quiver(0,0,0,ref_dir_cartesian[0],ref_dir_cartesian[1],ref_dir_cartesian[2],color="blue") # reference
ax1.quiver(0,0,0,final_dir_cartesian[0],final_dir_cartesian[1],final_dir_cartesian[2],color="red") # final

plt.xlim(-2.5, 2.5)
plt.ylim(-2.5, 2.5)
ax1.set_zlim(-2.5,2.5)
ax1.set_xlabel("Unity X / Cartesian X")
ax1.set_ylabel("Unity Z / Cartesian Y")
ax1.set_zlabel("Unity Y / Cartesian Z")

ax1.scatter(0,0,0,marker=".",color="black",s=500)

plt.show()

Version 1b - Angle differences with sphere coordinates
Horizontal angles:
	Input: -45.00000000000001
	Reference: 180.0
	Final: 135.00000000000003
Vertical angles:
	Input: 1.2074182697257333e-06
	Reference:-45.00000000000001
	Final: 44.999999999999986
Vectors:
	Input: [-1, 1, 0]
	Reference: [0, -1, -1]
	Final: (0.5, -0.5000000000000001, 0.7071067811865475)


In [4]:
# %%timeit

# Version 1a
#
# Basic idea
# Difference of elevation angle between reference and input 
# Difference of horizontal angle between reference and input 
# Apply angle difference to forward direction 
# Is likely not the same as rotating reference direction to forward direction 
# and applying same rotations to input  
print("Version 1a - Angle differences")

fig = plt.figure(figsize=(15,10))
fig.canvas.manager.set_window_title("Version 1a - Angle differences")
ax1 = fig.add_subplot(1,1,1, projection='3d')

# Input and reference 
#inp_dir_unity = np.array([1,-0.5,0.1])
#ref_dir_unity = np.array([-0.2,1,0.8])
inp_dir_unity = np.array([-1,0,1])
ref_dir_unity = np.array([0,-1,-1])

# To cartesian from Unity 
inp_dir_cartesian = unity_pts_to_right_handed_cartesian_coords([inp_dir_unity])[0]
ref_dir_cartesian = unity_pts_to_right_handed_cartesian_coords([ref_dir_unity])[0]

# Calculate thetas, can only use horizontal, since vertical depends on different plane than projected one 
inp_horizontal, _ = calculate_thetas_from_unity_coords(inp_dir_unity,ret_degree=True)
ref_horizontal, _ = calculate_thetas_from_unity_coords(ref_dir_unity,ret_degree=True)

# Calculate vertical angle offset as angle between direction vector and projection on x,y plane (cartesian)
inp_vertical = angle_between(inp_dir_cartesian,[inp_dir_cartesian[0],inp_dir_cartesian[1],0])
ref_vertical = angle_between(ref_dir_cartesian,[ref_dir_cartesian[0],ref_dir_cartesian[1],0])


# if z (cartesian) is negative, vertical angle is negative 
if inp_dir_cartesian[2] < 0:
    inp_vertical *= -1
if ref_dir_cartesian[2] < 0:
    ref_vertical *= -1


# Calculate angle offsets 
offset_horizontal = inp_horizontal - ref_horizontal
offset_vertical = inp_vertical - ref_vertical


print("Input horizontal: " + str(inp_horizontal))
print("Reference horizontal: " + str(ref_horizontal))
print("Offset horizontal: " + str(offset_horizontal)) # rotate around Unity y, Right-Handed cartesian z
print("Input vertical: " + str(inp_vertical))
print("Reference vertical: " + str(ref_vertical))
print("Offset vertical: " + str(offset_vertical)) # rotate around Unity/ Right-Handed cartesian x 

# calculate rot matrices
# axis are for right-handed coordinate system
# angle inverted for horizontal
horizontal_rot_mat = calculate_rotation_matrix(rotate_around_axis="z", angle= -1 * offset_horizontal, angle_in_degree=True)
vertical_rot_mat = calculate_rotation_matrix(rotate_around_axis="x", angle=offset_vertical, angle_in_degree=True)


# apply calculated relative rotations to forward Unity vector (0,0,1) (in right-handed cartesian (0,1,0)) to get final direction 
# first vertical, then horizontal! Rotation axis must align 
final_dir_unity = np.array([0,0,1])
final_dir_cartesian = unity_pts_to_right_handed_cartesian_coords([final_dir_unity])[0]
final_dir_cartesian = np.dot(vertical_rot_mat, final_dir_cartesian)
final_dir_cartesian = np.dot(horizontal_rot_mat,final_dir_cartesian)

# Verify final direction has offset values 
final_vertical = angle_between(final_dir_cartesian,[final_dir_cartesian[0],final_dir_cartesian[1],0]) # angle between vec and projection on xy-plane (cartesian)
final_horizontal = angle_between([final_dir_cartesian[0],final_dir_cartesian[1],0],[0,1,0]) # angle between projection on xy-plane and forward vector (cartesian)
print("---")
print("Final horizontal (should match offset horizontal): " + str(final_horizontal)) 
print("Final vertical (should match offset vertical): " + str(final_vertical))



# Plot
ax1.quiver(0,0,0,2,0,0,color="black") # axis
ax1.quiver(0,0,0,0,2,0,color="black") # axis
ax1.quiver(0,0,0,0,0,2,color="black") # axis
ax1.quiver(0,0,0,inp_dir_cartesian[0],inp_dir_cartesian[1],inp_dir_cartesian[2],color="green") # input
ax1.quiver(0,0,0,ref_dir_cartesian[0],ref_dir_cartesian[1],ref_dir_cartesian[2],color="blue") # reference
ax1.quiver(0,0,0,final_dir_cartesian[0],final_dir_cartesian[1],final_dir_cartesian[2],color="red") # final

plt.xlim(-2.5, 2.5)
plt.ylim(-2.5, 2.5)
ax1.set_zlim(-2.5,2.5)
ax1.set_xlabel("Unity X / Cartesian X")
ax1.set_ylabel("Unity Z / Cartesian Y")
ax1.set_zlabel("Unity Y / Cartesian Z")

ax1.scatter(0,0,0,marker=".",color="black",s=500)

plt.show()

Version 1a - Angle differences
Input horizontal: -45.0
Reference horizontal: 180.0
Offset horizontal: -225.0
Input vertical: 1.2074182697257333e-06
Reference vertical: -45.00000000000001
Offset vertical: 45.00000120741828
---
Final horizontal (should match offset horizontal): 135.0
Final vertical (should match offset vertical): 45.00000120741827


In [5]:
print(1/0)

ZeroDivisionError: division by zero

## Experimental & Untested below

In [ ]:

fig = plt.figure(figsize=(15,10))

ax1 = fig.add_subplot(1,1,1, projection='3d')

# from x,y,z to x,y,z
input_pts = np.array([[0,0,0,1,-1,-1]])
              #[0,0,0,1,-1,-1]])



# Use rotate from vector to vector method 

from_vec = input_pts[0][3:]
print("From vec pre scaling: " + str(from_vec))

from_vec = from_vec / np.linalg.norm(from_vec)
to_vec = np.array([0,0,1])
rot_mat_align = rotate_align(from_vec,to_vec)
print(rot_mat_align)

pts_new = np.dot(rot_mat_align,input_pts[0][3:])





print("From vec scaled: " + str(from_vec))
print(to_vec)

ax1.quiver(input_pts[:,0],input_pts[:,1],input_pts[:,2],input_pts[:,3],input_pts[:,4],input_pts[:,5])
ax1.quiver(0,0,0,pts_new[0],pts_new[1],pts_new[2],color="green")
plt.xlim(-2, 2)
plt.ylim(-2, 2)
ax1.set_zlim(0,2)
ax1.set_xlabel("x")
ax1.set_ylabel("y")
ax1.set_zlabel("z")

ax1.scatter(0,0,0,marker=".",color="red",s=500)

plt.show()




In [ ]:
## Offsets 

fig = plt.figure(figsize=(15,10))

ax1 = fig.add_subplot(1,1,1, projection='3d')


first_vec = np.array([[0,0,0,1,1,1]])
second_vec = np.array([[0,0,0,0,1,1]])
offset_one = np.array([first_vec[0] - second_vec[0]])

theta_z = 180 / 360 * 2 * np.pi 
rot_mat_z = np.array([
    [np.cos(theta_z),-np.sin(theta_z),0],
    [np.sin(theta_z),np.cos(theta_z),0],
    [0,0,1]
])


pts_new = np.dot(rot_mat_z, first_vec[0][3:])
third_vec = np.array([[0,0,0,pts_new[0],pts_new[1],pts_new[2]]]) # np.array([[0,0,0,-1,-1,-1]])
pts_new = np.dot(rot_mat_z, second_vec[0][3:])
fourth_vec = np.array([[0,0,0,pts_new[0],pts_new[1],pts_new[2]]]) # np.array([[0,0,0,0,-1,-1]])

print(third_vec)
offset_two = np.array([third_vec[0] - fourth_vec[0]])

ax1.quiver(first_vec[:,0],first_vec[:,1],first_vec[:,2],first_vec[:,3],first_vec[:,4],first_vec[:,5], color = "green")
ax1.quiver(second_vec[:,0],second_vec[:,1],second_vec[:,2],second_vec[:,3],second_vec[:,4],second_vec[:,5], color = "green")
ax1.quiver(offset_one[:,0],offset_one[:,1],offset_one[:,2],offset_one[:,3],offset_one[:,4],offset_one[:,5], color = "red")

ax1.quiver(third_vec[:,0],third_vec[:,1],third_vec[:,2],third_vec[:,3],third_vec[:,4],third_vec[:,5], color = "blue")
ax1.quiver(fourth_vec[:,0],fourth_vec[:,1],fourth_vec[:,2],fourth_vec[:,3],fourth_vec[:,4],fourth_vec[:,5], color = "blue")
ax1.quiver(offset_two[:,0],offset_two[:,1],offset_two[:,2],offset_two[:,3],offset_two[:,4],offset_two[:,5], color = "yellow")


plt.xlim(-2, 2)
plt.ylim(-2, 2)
ax1.set_zlim(0,2)
ax1.set_xlabel("x")
ax1.set_ylabel("y")
ax1.set_zlabel("z")

ax1.scatter(0,0,0,marker=".",color="red",s=500)

plt.show()






In [ ]:
## 
## More rotations 


# Rotation around axis, right-hand rule 
theta_x = 90 / 360 * np.pi * 2 
theta_y = 90 / 360 * np.pi * 2 
theta_z = 45 / 360 * np.pi * 2 

rot_mat_x = np.array([
    [1,0,0],
    [0,np.cos(theta_x),-np.sin(theta_x)],
    [0,np.sin(theta_x),np.cos(theta_x)]
])
rot_mat_y = np.array([
    [np.cos(theta_y),0,np.sin(theta_y)],
    [0,1,0],
    [-np.sin(theta_y),0,np.cos(theta_y)]
])
rot_mat_z = np.array([
    [np.cos(theta_z),-np.sin(theta_z),0],
    [np.sin(theta_z),np.cos(theta_z),0],
    [0,0,1]
])

#pts_new = np.dot(rot_mat_z, input_pts[0][3:])
#pts_new = np.dot(rot_mat_x, pts_new)
#pts_new = np.dot(rot_mat_y, pts_new)



'''
# Create Rotation Matrix from rotation vector, perform angle shifts in rotation vector simultaneously
# Right hand rule 
rotvec = (-1 * offset_vertical,0,offset_horizontal)
#rotvec = (45,45,0) ## TODO 
r = R.from_rotvec(rotvec)
rot_mat_combined = r.as_matrix()
final_dir_unity = np.array([0,0,1])
final_dir_cartesian = unity_pts_to_right_handed_cartesian_coords([final_dir_unity])[0]
final_dir_cartesian = np.dot(rot_mat_combined, final_dir_cartesian)
final_dir_cartesian = np.array([0,0,0,final_dir_cartesian[0],final_dir_cartesian[1],final_dir_cartesian[2]])
'''


# Create Rotation Matrix from rotation vector, perform angle shifts in rotation vector simultaneously
# Right hand rule 
theta_x = 90 / 360 * 2 * np.pi
theta_z = 90 / 360 * 2 * np.pi
theta_y = 45 / 360 * 2 * np.pi 

rotvec = (theta_x,0,theta_z)

r = R.from_rotvec(rotvec)
rot_mat_combined = r.as_matrix()

#pts_new = np.dot(rot_mat_combined,input_pts[0][3:])

In [ ]:
# Other stuff 

# sth else https://math.stackexchange.com/questions/180418/calculate-rotation-matrix-to-align-vector-a-to-vector-b-in-3d/897677#897677

'''
def rotate_align(v1,v2):
    
    #Untested!
    #Credit: https://gist.github.com/kevinmoran/b45980723e53edeb8a5a43c49f134724
    
    
    axis = np.cross(v1,v2)
    
    cosA = np.dot(v1,v2)
    
    k = 1.0 / (1.0 + cosA)

    
    result = np.array([[(axis[0] * axis[0] * k) + cosA, \
                        (axis[1] * axis[0] * k) - axis[2], \
                        (axis[2] * axis[0] * k) + axis[1]], \
                        [(axis[0] * axis[1] * k) + axis[2], \
                        (axis[1] * axis[1] * k) + cosA,     \
                        (axis[2] * axis[1] * k) - axis[0]], \
                        [(axis[0] * axis[2] * k) - axis[1], \
                        (axis[1] * axis[2] * k) + axis[0],  \
                        (axis[2] * axis[2] * k) + cosA]])
    return result
'''
